In [65]:
%matplotlib inline
%config Application.log_level="INFO"

import matplotlib
import numpy as np
import mxnet as mx

import matplotlib.pyplot as plt

In [66]:
import logging
logging.basicConfig()
logger = logging.getLogger('Something')
logger.setLevel(logging.DEBUG)

In [78]:
def load_data(file):
    data = np.loadtxt(file, delimiter=',', skiprows=1)
    return data

def split_data(data, ratio):
    rows = data.shape[0]
    rows = int(rows * ratio)
    train_data, eval_data = np.split(data, [rows,], axis=0)
    return train_data, eval_data

def data_to_iter(data, batch_size=10):
    y, x = np.split(data, [1], axis=1)
    
    y = y.flatten()
    
    data_iter = mx.io.NDArrayIter(
        data=x,
        label=y,
        batch_size=batch_size
    )
    
    return data_iter


In [74]:
def build_model(context):
    data = mx.sym.Variable('data')
    data = mx.sym.reshape(data=data, shape=[-1, 28, 28])
    
    net = mx.sym.FullyConnected(data, num_hidden=64)
    net = mx.sym.Activation(net, act_type='relu')
    net = mx.sym.FullyConnected(net, num_hidden=10)
    net = mx.sym.SoftmaxOutput(net, preserve_shape=True, name='softmax')
    
    model = mx.mod.Module(
        symbol=net,
        context=context,
        data_names=['data'],
        label_names=['softmax_label'],
        logger=logger,
    )
    
    return model

In [69]:
data = load_data('train.csv')


In [79]:
train_data, eval_data = split_data(data, 0.8)

train_iter = data_to_iter(train_data)
eval_iter = data_to_iter(eval_data)

print(train_data.shape)
print(eval_data.shape)

(33600, 785)
(8400, 785)


In [80]:
model = build_model(mx.cpu())


In [85]:
model.fit(
    train_data=train_iter,
    eval_data=eval_iter,
    optimizer='sgd',
    optimizer_params={'learning_rate':0.001},
    eval_metric=['acc'],
    num_epoch=100,
    force_init=True,
    force_rebind=True,
)

INFO:Something:Epoch[0] Train-accuracy=0.904048
INFO:Something:Epoch[0] Time cost=2.202
INFO:Something:Epoch[0] Validation-accuracy=0.937976
INFO:Something:Epoch[1] Train-accuracy=0.946280
INFO:Something:Epoch[1] Time cost=2.155
INFO:Something:Epoch[1] Validation-accuracy=0.953095
INFO:Something:Epoch[2] Train-accuracy=0.958810
INFO:Something:Epoch[2] Time cost=2.238
INFO:Something:Epoch[2] Validation-accuracy=0.948095
INFO:Something:Epoch[3] Train-accuracy=0.964345
INFO:Something:Epoch[3] Time cost=2.177
INFO:Something:Epoch[3] Validation-accuracy=0.955119
INFO:Something:Epoch[4] Train-accuracy=0.967589
INFO:Something:Epoch[4] Time cost=2.247
INFO:Something:Epoch[4] Validation-accuracy=0.953929
INFO:Something:Epoch[5] Train-accuracy=0.969792
INFO:Something:Epoch[5] Time cost=2.201
INFO:Something:Epoch[5] Validation-accuracy=0.960357
INFO:Something:Epoch[6] Train-accuracy=0.973958
INFO:Something:Epoch[6] Time cost=2.298
INFO:Something:Epoch[6] Validation-accuracy=0.959405
INFO:Somethin

KeyboardInterrupt: 

In [84]:
model.score(eval_iter, ['loss', 'acc'])

[('loss', 0.10000000002270654), ('accuracy', 0.95)]

In [53]:
x = model.predict(eval_iter)

In [64]:
print(x[3])
print(train_data[3, 784])


[9.9990273e-01 7.0241281e-06 5.1534157e-06 2.3766444e-05 4.7366452e-06
 9.6804843e-06 1.3643874e-05 8.5173388e-06 1.9752706e-05 5.1248130e-06]
<NDArray 10 @cpu(0)>
0.0
